In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Final_Project/py-files')

In [ ]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 876.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.7 MB/s eta 0:00:00


In [ ]:
# imports
import numpy as np
import pandas as pd
from langchain.vectorstores import FAISS

# Setting up your LLM

In [ ]:
import os
from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('LLM_TOKEN')

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

# This info's at the top of each HuggingFace model page
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(repo_id = hf_model)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Creating vectors with embeddings

[Embeddings](https://python.langchain.com/docs/integrations/text_embedding) are a fancy way of saying we turn words into numbers that computers can understand. Each word gets its own unique code, based on its meaning and relationship to other words. The list of numbers produced is known as a vector. Vectors allow us to compare text and find chunks that contain similar information.

Different embedding models encode words and meanings in different ways, and finding the right one can be tricky. We're using open-source models from HuggingFace, who even have a handy [leaderboard of embeddings](https://huggingface.co/spaces/mteb/leaderboard) on their website. Just browse the options and see which one speaks your language (literally!).
> As we are doing a retrieval project, click on the `Retrieval` tab of the leaderboard to see the best embeddings for retrieval tasks.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Test Embedding

In [ ]:
test_text = "Why do data scientists make great comedians? They're always trying to make ANOVA pun"
query_result = embeddings.embed_query(test_text)
query_result[:10]

[0.009409730322659016,
 -0.023806262761354446,
 -0.012127556838095188,
 0.036123793572187424,
 -0.03382446989417076,
 -0.07974203675985336,
 0.07004599273204803,
 0.0746554508805275,
 0.040141817182302475,
 0.044190675020217896]

In [ ]:
characters = len(test_text)
dimensions = len(query_result)
print(f"The {characters} character sentence was transformed into a {dimensions} dimension vector")

The 84 character sentence was transformed into a 384 dimension vector


#Load FAISS Vector Store

In [ ]:
# load Frankenstein vector_store
path = '/content/drive/MyDrive/Final_Project/data/faiss_index_book2_frankenstein'
book2_vs = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)

In [ ]:
# load 1984 vector_store
path = '/content/drive/MyDrive/Final_Project/data/faiss_index_book3_1984'
book3_vs = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)

## Test the Vector Store

In [ ]:
results = book2_vs.similarity_search("What did Frankenstein create?")

In [ ]:
print(results[0].page_content)

His tale is connected and told with an appearance of the simplest truth,
yet I own to you that the letters of Felix and Safie, which he showed me,
and the apparition of the monster seen from our ship, brought to me a
greater conviction of the truth of his narrative than his asseverations,
however earnest and connected. Such a monster has, then, really existence!
I cannot doubt it, yet I am lost in surprise and admiration. Sometimes I
endeavoured to gain from Frankenstein the particulars of his
creature’s formation, but on this point he was impenetrable.

“Are you mad, my friend?” said he. “Or whither does your
senseless curiosity lead you? Would you also create for yourself and the
world a demoniacal enemy? Peace, peace! Learn my miseries and do not seek
to increase your own.”


In [ ]:
results = book3_vs.similarity_search("What are the three super states?")

In [ ]:
print(results[0].page_content)

The splitting up of the world into three great super-states was an event
which could be and indeed was foreseen before the middle of the twentieth
century. With the absorption of Europe by Russia and of the British Empire
by the United States, two of the three existing powers, Eurasia and
Oceania, were already effectively in being. The third, Eastasia, only
emerged as a distinct unit after another decade of confused fighting. The
frontiers between the three super-states are in some places arbitrary, and
in others they fluctuate according to the fortunes of war, but in general
they follow geographical lines. Eurasia comprises the whole of the northern
part of the European and Asiatic land-mass, from Portugal to the Bering


# Definition of similarity_search_with_relevance() function
This function gives the relevance of the question w.r.t. the corpus.

In [ ]:
# return search results (list of documents), a pandas DataFrame containing the distances, nn_distances and quality numbers,
# and the relevance of the query with respect to the vector_store.
def similarity_search_with_relevance(query, vector_store, k=4, omit_first = False):
  def get_quality(distance_0, distance_1):
    #distance_0: distance of query_vector to the vector you want to calculate the quality for
    #distance_1: distance of vector you want to calculate the quality for to its nearest neighbor
    if distance_1 > distance_0: # this can happen when the query vector is not in the vector store and closer to its NN than the nearest neighbor of the NN
      return 1.0
    return distance_1/distance_0



  nearest_neighbors = vector_store.similarity_search_with_score(query, k=k)
  if omit_first:
    nearest_neighbors = nearest_neighbors[1:]

  #get doc_list
  doc_list = [doc for doc, score in nearest_neighbors]

  # turn nearest_neighbors into two lists; the first containing doc.page_content and the second containing distance
  doc_page_content_list, distances = [doc.page_content for doc in doc_list], [np.sqrt(score) for doc, score in nearest_neighbors]

  # for each content in doc_content_list, do a search and get its nearest neighbor score
  nn_scores = [vector_store.similarity_search_with_score(page_content, k=2)[1][1] for page_content in doc_page_content_list]
  nn_distances = np.sqrt(nn_scores)

  # make a DataFrame containing page_content, distance, nn_distance
  results_df = pd.DataFrame({'page_content': doc_page_content_list, 'distance':distances, 'nn_distance':nn_distances})

  # for each row in results_df, calculate the quality
  qualities = [get_quality(row.distance, row.nn_distance) for index, row in results_df.iterrows()]

  # add column with qualities to results_df
  results_df['quality'] = qualities

  query_relevance = results_df.quality.mean()

  return {'docs':doc_list, 'results':results_df, 'relevance':query_relevance}

# Definition of similarity_search_with_weighted_relevance
This function calculates a weighted relevance number by multiplying the quality-ratio of each nearest neighbor with its relevance, where relevance is returned by the function similarity_search_with_relevance().   

In [ ]:
def similarity_search_with_weighted_relevance(query, vector_store, k=4):
  search_results = similarity_search_with_relevance(query, vector_store, k=k, omit_first = False)
  # for each nearest neighbor of the query determine its relevance (not weighted relevance!!!)
  relevances = [similarity_search_with_relevance(doc.page_content, vector_store, k=k+1, omit_first = True)['relevance'] for doc in search_results['docs']]
  qualities = search_results['results']['quality']
  weighted_qualities = relevances*qualities
  weighted_relevance = weighted_qualities.mean()
  return {'docs':search_results['docs'], 'results':search_results['results'], 'weighted_relevance':weighted_relevance}

In [ ]:
t = pd.Series([1,2,3,4])
t.mean()

2.5

# Test Relevance and Weighted Relevance function with different Queries

# Questions

In [ ]:
question1 = 'What did Frankenstein create?'
question2 = 'Who is Henry Clerval?'
question3 = 'What are the three super states?'
question4 = "What happens in room 101?"
question5 = 'What courses does WBS Coding School offer?'
question6 = 'Who is Big Brother?'

#Question 1

In [ ]:
question = question1
print('corpus: Frankenstein\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book2_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book2_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: Frankenstein

Question: What did Frankenstein create?

relevance = 0.880194286507724
weighted relevance = 0.7346023088130982

His tale is connected and told with an appearance of the simplest truth,
yet I own to you that the letters of Felix and Safie, which he showed me,
and the apparition of the monster seen from our ship, brought to me a
greater conviction of the truth of his narrative than his asseverations,
however earnest and connected. Such a monster has, then, really existence!
I cannot doubt it, yet I am lost in surprise and admiration. Sometimes I
endeavoured to gain from Frankenstein the particulars of his
creature’s formation, but on this point he was impenetrable.

“Are you mad, my friend?” said he. “Or whither does your
senseless curiosity lead you? Would you also create for yourself and the
world a demoniacal enemy? Peace, peace! Learn my miseries and do not seek
to increase your own.”


In [ ]:
question = question1
print('corpus: 1984\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book3_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book3_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: 1984

Question: What did Frankenstein create?

relevance = 0.7445610047959332
weighted relevance = 0.6337192169267537

darkness. With a deadly effort, like wrenching a piece out of his own
brain, he could even have dragged the thing into the open. He always woke
up without discovering what it was: but somehow it was connected with what
Julia had been saying when he cut her short.


# Question 2

In [ ]:
question = question2
print('corpus: Frankenstein\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book2_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book2_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: Frankenstein

Question: Who is Henry Clerval?

relevance = 0.848428998525216
weighted relevance = 0.7718371757048416

On the birth of a second son, my junior by seven years, my parents gave
up entirely their wandering life and fixed themselves in their native
country. We possessed a house in Geneva, and a _campagne_ on Belrive,
the eastern shore of the lake, at the distance of rather more than a
league from the city. We resided principally in the latter, and the
lives of my parents were passed in considerable seclusion. It was my
temper to avoid a crowd and to attach myself fervently to a few. I was
indifferent, therefore, to my school-fellows in general; but I united
myself in the bonds of the closest friendship to one among them. Henry
Clerval was the son of a merchant of Geneva. He was a boy of singular
talent and fancy. He loved enterprise, hardship, and even danger for


In [ ]:
question = question2
print('corpus: 1984\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book3_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book3_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: 1984

Question: Who is Henry Clerval?

relevance = 0.7156996623362984
weighted relevance = 0.5893235825785677

He was rolling down a mighty corridor, a kilometre wide, full of glorious,
golden light, roaring with laughter and shouting out confessions at the
top of his voice. He was confessing everything, even the things he had
succeeded in holding back under the torture. He was relating the entire
history of his life to an audience who knew it already. With him were the
guards, the other questioners, the men in white coats, O'Brien, Julia,
Mr Charrington, all rolling down the corridor together and shouting with
laughter. Some dreadful thing which had lain embedded in the future had
somehow been skipped over and had not happened. Everything was all right,
there was no more pain, the last detail of his life was laid bare,
understood, forgiven.


# Question 3

In [ ]:
question = question3
print('corpus: Frankenstein\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book2_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book2_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: Frankenstein

Question: What are the three super states?

relevance = 0.6788363948666276
weighted relevance = 0.5924728964501875

visible, its broad disk just skirting the horizon and diffusing a
perpetual splendour. There—for with your leave, my sister, I will put
some trust in preceding navigators—there snow and frost are banished;
and, sailing over a calm sea, we may be wafted to a land surpassing in
wonders and in beauty every region hitherto discovered on the habitable
globe. Its productions and features may be without example, as the
phenomena of the heavenly bodies undoubtedly are in those undiscovered
solitudes. What may not be expected in a country of eternal light? I
may there discover the wondrous power which attracts the needle and may
regulate a thousand celestial observations that require only this
voyage to render their seeming eccentricities consistent for ever. I


In [ ]:
question = question3
print('corpus: 1984\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book3_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book3_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: 1984

Question: What are the three super states?

relevance = 0.8693867664985613
weighted relevance = 0.7765255911720315

The splitting up of the world into three great super-states was an event
which could be and indeed was foreseen before the middle of the twentieth
century. With the absorption of Europe by Russia and of the British Empire
by the United States, two of the three existing powers, Eurasia and
Oceania, were already effectively in being. The third, Eastasia, only
emerged as a distinct unit after another decade of confused fighting. The
frontiers between the three super-states are in some places arbitrary, and
in others they fluctuate according to the fortunes of war, but in general
they follow geographical lines. Eurasia comprises the whole of the northern
part of the European and Asiatic land-mass, from Portugal to the Bering


# Question 4

In [ ]:
question = question4
print('corpus: Frankenstein\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book2_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book2_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: Frankenstein

Question: What happens in room 101?

relevance = 0.6094389414363691
weighted relevance = 0.5410778311076314

Chapter 11


In [ ]:
question = question4
print('corpus: 1984\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book3_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book3_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: 1984

Question: What happens in room 101?

relevance = 0.7827691972078202
weighted relevance = 0.6757323703170669

It was bigger than most of the cells he had been in. But he hardly noticed
his surroundings. All he noticed was that there were two small tables
straight in front of him, each covered with green baize. One was only a
metre or two from him, the other was further away, near the door. He was
strapped upright in a chair, so tightly that he could move nothing, not
even his head. A sort of pad gripped his head from behind, forcing him to
look straight in front of him.

For a moment he was alone, then the door opened and O'Brien came in.

'You asked me once,' said O'Brien, 'what was in Room 101. I told you that
you knew the answer already. Everyone knows it. The thing that is in
Room 101 is the worst thing in the world.'


# Question 5

In [ ]:
question = question5
print('corpus: Frankenstein\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book2_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book2_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: Frankenstein

Question: What courses does WBS Coding School offer?

relevance = 0.5502429631925726
weighted relevance = 0.4692618825137779

START: FULL LICENSE

THE FULL PROJECT GUTENBERG LICENSE

PLEASE READ THIS BEFORE YOU DISTRIBUTE OR USE THIS WORK

To protect the Project Gutenberg™ mission of promoting the free
distribution of electronic works, by using or distributing this work
(or any other work associated in any way with the phrase “Project
Gutenberg”), you agree to comply with all the terms of the Full
Project Gutenberg™ License available with this file or online at
www.gutenberg.org/license.

Section 1. General Terms of Use and Redistributing Project Gutenberg™
electronic works


In [ ]:
question = question5
print('corpus: 1984\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book3_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book3_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: 1984

Question: What courses does WBS Coding School offer?

relevance = 0.641932704081316
weighted relevance = 0.5091851013536812

THE END




Project Gutenberg Australia


# Question 6

In [ ]:
question = question6
print('corpus: Frankenstein\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book2_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book2_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: Frankenstein

Question: Who is Big Brother?

relevance = 0.654053202050111
weighted relevance = 0.5529374540061427

“I do not know what you mean,” replied my brother, in accents of
wonder, “but to us the discovery we have made completes our misery. No
one would believe it at first; and even now Elizabeth will not be
convinced, notwithstanding all the evidence. Indeed, who would credit
that Justine Moritz, who was so amiable, and fond of all the family,
could suddenly become so capable of so frightful, so appalling a crime?”

“Justine Moritz! Poor, poor girl, is she the accused? But it is
wrongfully; every one knows that; no one believes it, surely, Ernest?”


In [ ]:
question = question6
print('corpus: 1984\n')
print('Question: '+question+'\n')
results  = similarity_search_with_relevance(question, book3_vs)
weighted_results  = similarity_search_with_weighted_relevance(question, book3_vs)
relevance = results['relevance']
weighted_relevance = weighted_results['weighted_relevance']
print(f'relevance = {relevance}')
print(f'weighted relevance = {weighted_relevance}')
print()
print(results['results'].page_content[0])

corpus: 1984

Question: Who is Big Brother?

relevance = 0.8144148511811904
weighted relevance = 0.6954839615655598

who was thirty-nine and had a varicose ulcer above his right ankle, went
slowly, resting several times on the way. On each landing, opposite the
lift-shaft, the poster with the enormous face gazed from the wall. It was
one of those pictures which are so contrived that the eyes follow you about
when you move. BIG BROTHER IS WATCHING YOU, the caption beneath it ran.
